```{contents}
```
## **Review Node in LangGraph**

A **Review Node** in LangGraph is a specialized control node used to **evaluate, validate, critique, or approve the output of previous nodes** before allowing execution to proceed.
It acts as a **quality-control gate** inside the execution graph.

This enables **self-correction, reliability, safety, and production-grade robustness**.

---

### **1. Motivation & Intuition**

LLM systems are **probabilistic** and may produce:

* incorrect reasoning
* unsafe content
* low-quality answers
* policy violations

A Review Node inserts an explicit **evaluation step** before downstream actions occur.

```
Generate → Review → (Accept | Revise | Reject)
```

---

### **2. Functional Role in the Graph**

| Role               | Purpose                  |
| ------------------ | ------------------------ |
| Quality Gate       | Blocks bad outputs       |
| Safety Filter      | Enforces constraints     |
| Correction Trigger | Enables self-improvement |
| Audit Checkpoint   | Stores reviewed results  |

---

### **3. Where Review Nodes Are Used**

* High-risk decisions (finance, medical, legal)
* Autonomous agents
* Multi-agent coordination
* Tool execution verification
* Compliance pipelines

---

### **4. Core Design Pattern**

```
Producer Node → Review Node → Router → Next Step
```

The Review Node does **not** produce the final answer.
It produces a **judgment** about the answer.

---

### **5. State Schema Example**

```python
class State(TypedDict):
    draft: str
    review: str
    approved: bool
```

---

### **6. Implementing a Review Node**

```python
def generate(state):
    return {"draft": "Proposed solution text"}

def review(state):
    text = state["draft"]
    if "unsafe" in text:
        return {"review": "Rejected: unsafe content", "approved": False}
    return {"review": "Approved", "approved": True}
```

---

### **7. Routing Based on Review**

```python
def route_after_review(state):
    if state["approved"]:
        return "final"
    return "revise"

builder.add_conditional_edges("review", route_after_review, {
    "final": "final",
    "revise": "generate"
})
```

This creates a **self-correcting loop**.

---

### **8. Full Minimal Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    draft: str
    review: str
    approved: bool

def generate(state):
    return {"draft": "My answer"}

def review_node(state):
    ok = len(state["draft"]) > 5
    return {"approved": ok, "review": "ok" if ok else "too short"}

def final(state):
    return {"final": state["draft"]}

builder = StateGraph(State)
builder.add_node("generate", generate)
builder.add_node("review", review_node)
builder.add_node("final", final)

builder.set_entry_point("generate")
builder.add_edge("generate", "review")

builder.add_conditional_edges("review",
    lambda s: "final" if s["approved"] else "generate",
    {"final": "final", "generate": "generate"}
)

builder.add_edge("final", END)

graph = builder.compile()
graph.invoke({})
```

---

### **9. Review Node Variants**

| Variant            | Function                   |
| ------------------ | -------------------------- |
| Rule-Based Review  | Deterministic validation   |
| LLM-Based Review   | Semantic critique          |
| Hybrid Review      | LLM + rules                |
| Human Review       | Manual approval            |
| Multi-Agent Review | Consensus-based validation |

---

### **10. Production Patterns**

| Pattern             | Description             |
| ------------------- | ----------------------- |
| Reflection Loop     | Draft → Review → Revise |
| Approval Gate       | Block unsafe outputs    |
| Self-Healing Agent  | Automatic correction    |
| Compliance Workflow | Enforced governance     |
| Autonomous Systems  | Stability control       |

---

### **11. Why Review Nodes Are Critical**

Without Review Nodes:

* errors propagate
* hallucinations persist
* unsafe actions execute

With Review Nodes:

* systems become **self-correcting**
* output quality improves
* trust & reliability increase

---

### **12. Mental Model**

> A Review Node is a **checkpoint of intelligence**
> that prevents your system from trusting itself blindly.


### Demonstration

In [1]:
from typing import TypedDict

class State(TypedDict):
    draft: str
    review: str
    approved: bool


In [3]:
def generate(state):
    if "draft" not in state:
        return {"draft": "Short"}
    return {"draft": state["draft"] + " improved"}

def review(state):
    ok = len(state["draft"]) >= 10
    return {
        "approved": ok,
        "review": "Approved" if ok else "Too short"
    }

def finalize(state):
    return {"final_answer": state["draft"]}


In [ ]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("review", review)
builder.add_node("final", finalize)

builder.set_entry_point("generate")
builder.add_edge("generate", "review")

builder.add_conditional_edges(
    "review",
    lambda s: "final" if s["approved"] else "generate",
    {"final": "final", "generate": "generate"}
)

builder.add_edge("final", END)

graph = builder.compile()

In [ ]:
result = graph.invoke({})
print(result)

{'draft': 'Short improved', 'review': 'Approved', 'approved': True}
